In [ ]:
# Cargar el archivo parquet en un DataFrame llamado df_fleca
# Usamos pyarrow.parquet para poder acceder al schema y tipos de columnas del archivo parquet,
# lo que permite detectar y convertir tipos especiales como 'dbdate' antes de cargar a pandas.
import pandas as pd
import pyarrow.parquet as pq

parquet_file = '../data/raw/raw_data_bq_forecasting_20250630.parquet'
table = pq.read_table(parquet_file)
schema = table.schema

# Identificar columnas con tipo 'dbdate'
dbdate_cols = [field.name for field in schema if str(field.type) == 'dbdate']

# Convertir columnas 'dbdate' a string en el objeto Table de pyarrow
for col in dbdate_cols:
    table = table.set_column(table.schema.get_field_index(col), col, table.column(col).cast('string'))

# Convertir a pandas desactivando la metadata de pandas
df_fleca = table.to_pandas(ignore_metadata=True)

df_fleca.head()

In [56]:
# Cargar el archivo parquet en un DataFrame llamado df_fleca

# Usamos pyarrow.parquet para poder acceder al schema y tipos de columnas del archivo parquet,

# lo que permite detectar y convertir tipos especiales como 'dbdate' antes de cargar a pandas.
import pandas as pd
import pyarrow.parquet as pq  

parquet_file = '../data/raw/raw_data_bq_forecasting_20250630.parquet'
table = pq.read_table(parquet_file)
schema = table.schema

# Identificar columnas con tipo 'dbdate'
dbdate_cols = [field.name for field in schema if str(field.type) == 'dbdate']

# Convertir columnas 'dbdate' a string en el objeto Table de pyarrow
for col in dbdate_cols:
    table = table.set_column(table.schema.get_field_index(col), col, table.column(col).cast('string'))

# Convertir a pandas desactivando la metadata de pandas
df_fleca = table.to_pandas(ignore_metadata=True)

df_fleca.head()

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total
0,2025-06-30,T/192239,B1,EXTRA HIELO,CAFES,1.0,0.09,10.0,0.10
1,2025-06-22,T/190647,B1,EXTRA HIELO,CAFES,1.0,0.09,10.0,0.10
2,2025-06-23,T/190781,B1,EXTRA HIELO,CAFES,1.0,0.09,10.0,0.10
3,2025-06-22,T/190647,B1,BOLSA DE PLASTICO,VARIOS,1.0,0.14,10.0,0.15
4,2025-04-30,T/182453,B1,EXTRA HIELO,CAFES,1.0,0.18,10.0,0.20


In [159]:
# Valores nulos
print(df_fleca.isnull().sum())


fecha             0
n_factura         0
zona_de_venta     0
producto          0
familia           0
cantidad          0
base_imponible    0
tipo_IVA          0
total             0
is_closed         0
dtype: int64


In [58]:
# Mostrar los registro de valores nulos de la columna base imponible
print(df_fleca[df_fleca['base_imponible'].isnull()])



             fecha n_factura zona_de_venta               producto     familia  \
24670   2025-06-04   #VALUE!         SALON  PETIT ARTESANO (UNID)    BOLLERIA   
57562   2023-11-04  T/103357            B1                 TALLAT       CAFES   
57563   2023-11-08  T/104091            B1                 TALLAT       CAFES   
57564   2023-11-08  T/104096            B1              CAFE LLET       CAFES   
57565   2023-11-07  T/103854            B1    PETIT ARTESANO (Kg)    BOLLERIA   
...            ...       ...           ...                    ...         ...   
176975  2023-11-05  T/103522           LL5              CAFE LLET       CAFES   
176976  2023-11-08  T/104000           LL5         FLAUTI TONYINA  BOCADILLOS   
176977  2023-11-05  T/103562           LL5       BAGUETTE RUSTICA         PAN   
176978  2023-11-06  T/103769           LL5       7 CEREALS XORIÇO  BOCADILLOS   
176979  2023-11-05  T/103604           LL5    BRAZO PQ TRUFA/XOCO  PASTELERIA   

        cantidad  base_impo

In [59]:
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

In [158]:
# Asegurarse de que la columna 'fecha' no tenga valores nulos
df_fleca[df_fleca['fecha'].notnull()]  # Filtrar filas donde 'fecha' no es nula

# Convertir la columna de fecha a periodos mensuales
df['mes'] = df['fecha'].dt.to_period('M')

# Calcular valores nulos en la columna 'base_imponible' por mes
valores_nulos_por_mes = df[df['base_imponible'].isnull()].groupby('mes').size()

# Mostrar el resultado
print("Valores nulos por mes:")
print(valores_nulos_por_mes)

NameError: name 'df' is not defined

In [61]:
# Ver cuántos valores nulos hay por familia
df_nulos_familiaa = df_fleca[df_fleca.isnull().any(axis=1)].groupby("familia").size()
print("Valores nulos por familia:")
print(df_nulos_familiaa)

Valores nulos por familia:
familia
AÑADIDOS             11
BEBIDA               98
BEBIDAS              87
BOCADILLOS          329
BOLLERIA            427
CAFES               872
CERVEZAS             24
LICORES               8
PAN                 260
PASTELERIA           55
TES & INFUSIONES     44
TOSTADAS             21
VARIOS               12
dtype: int64


In [64]:
# Identificar los meses cercanos a noviembre 2023
meses_cercanos = ["2023-10", "2023-12"]

# Convertir a formato Periodo (mes-año) para facilitar la agrupación
df_fleca['mes'] = df_fleca['fecha'].dt.to_period("M")

# Imputar valores nulos en 'base_imponible', 'tipo_IVA' y 'total' usando la media de la categoría en meses cercanos
for columna in ['base_imponible', 'tipo_IVA', 'total']:
    for categoria in df_fleca['familia'].unique():
        for mes in meses_cercanos:
            # Filtrar la media de la categoría en los meses cercanos
            media_categoria = df_fleca[(df_fleca['familia'] == categoria) & (df_fleca['mes'] == mes)][columna].mean()
            
            # Verificar si la media es válida
            if not pd.isna(media_categoria):
                # Rellenar valores nulos solo en noviembre 2023 de esa categoría
                df_fleca.loc[
                    (df_fleca['familia'] == categoria) &
                    (df_fleca['mes'] == "2023-11") &
                    (df_fleca[columna].isnull()),
                    columna
                ] = media_categoria

# Si sigue habiendo valores nulos, usar la media anual de la categoría en 2023
for columna in ['base_imponible', 'tipo_IVA', 'total']:
    for categoria in df_fleca['familia'].unique():
        media_anual = df_fleca[(df_fleca['familia'] == categoria) & (df_fleca['mes'].dt.year == 2023)][columna].mean()
        
        # Verificar si la media anual es válida
        if not pd.isna(media_anual):
            df_fleca.loc[
                (df_fleca['familia'] == categoria) &
                (df_fleca['mes'] == "2023-11") &
                (df_fleca[columna].isnull()),
                columna
            ] = media_anual

# Verificar si quedan valores nulos
print("Valores nulos después de la imputación:")
print(df_fleca.isnull().sum())

Valores nulos después de la imputación:
fecha             0
n_factura         0
zona_de_venta     0
producto          0
familia           1
cantidad          0
base_imponible    1
tipo_IVA          0
total             1
mes               0
dtype: int64


In [65]:
# Eliminar cualquier fila que tenga un valor nulo en cualquier columna
df_fleca = df_fleca.dropna()

# Verificar que ya no haya valores nulos
print("Valores nulos después de eliminar la fila con nulos restantes:")
print(df_fleca.isnull().sum())


Valores nulos después de eliminar la fila con nulos restantes:
fecha             0
n_factura         0
zona_de_venta     0
producto          0
familia           0
cantidad          0
base_imponible    0
tipo_IVA          0
total             0
mes               0
dtype: int64


In [66]:
# Mostrar el total de filas de la familia "BEBIDAS" Y "BEBIDA" por separado
print(df_fleca[df_fleca['familia'] == 'BEBIDAS'].shape[0])
print(df_fleca[df_fleca['familia'] == 'BEBIDA'].shape[0])   


13138
17173


In [67]:
# Cambiar el nombre de las observaciones de la familia "BEBIDA" a "BEBIDAS"
df_fleca.loc[df_fleca['familia'] == 'BEBIDA', 'familia'] = 'BEBIDAS'
# Verificar que se han cambiado los nombres
print(df_fleca['familia'].unique())

['CAFES' 'VARIOS' 'AÑADIDOS' 'BOLLERIA' 'PAN' 'BEBIDAS' 'CERVEZAS'
 'BOCADILLOS' 'LICORES' 'TOSTADAS' 'TES & INFUSIONES' 'PASTELERIA' '7'
 'AIGU' 'AMERI' 'LLET' 'º' 'PETI' 'PETIT XOCO' '2']


In [68]:
# Contar los registros de la familia '7' 'AIGU'  'AMERI' 'LLET' 'º' None 'PETI' 'PETIT XOCO' '2']
print(df_fleca[df_fleca['familia'] == '7'].shape[0])
print(df_fleca[df_fleca['familia'] == 'AIGU'].shape[0])
print(df_fleca[df_fleca['familia'] == 'AMERI'].shape[0])  
print(df_fleca[df_fleca['familia'] == 'LLET'].shape[0])
print(df_fleca[df_fleca['familia'] == 'º'].shape[0])
print(df_fleca[df_fleca['familia'] == None].shape[0])
print(df_fleca[df_fleca['familia'] == 'PETI'].shape[0])
print(df_fleca[df_fleca['familia'] == 'PETIT XOCO'].shape[0])
print(df_fleca[df_fleca['familia'] == '2'].shape[0])          

1
1
1
2
1
0
1
1
1


In [69]:
#Eliminar los registros de la familia '7' 'AIGU'  'AMERI' 'LLET' 'º' None 'PETI' 'PETIT XOCO' '2']
df_fleca = df_fleca[~df_fleca['familia'].isin(['7', 'AIGU', 'AMERI', 'LLET', 'º', None, 'PETI', 'PETIT XOCO', '2'])]
# Verificar que se han eliminado los registros
print(df_fleca['familia'].unique())

['CAFES' 'VARIOS' 'AÑADIDOS' 'BOLLERIA' 'PAN' 'BEBIDAS' 'CERVEZAS'
 'BOCADILLOS' 'LICORES' 'TOSTADAS' 'TES & INFUSIONES' 'PASTELERIA']


In [70]:
# Nos quedamos solo con las columnas de interés
df_fleca = df_fleca[['fecha', 'familia', 'base_imponible']]
df_fleca.head()

,fecha,familia,base_imponible
0,2025-06-30,CAFES,0.09
1,2025-06-22,CAFES,0.09
2,2025-06-23,CAFES,0.09
3,2025-06-22,VARIOS,0.14
4,2025-04-30,CAFES,0.18


In [71]:
# Verificar el tipo de datos de las columnas
df_fleca.info()


<class 'pandas.core.frame.DataFrame'>
Index: 309315 entries, 0 to 309325
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           309315 non-null  datetime64[ns]
 1   familia         309315 non-null  object        
 2   base_imponible  309315 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 9.4+ MB


In [72]:
#Cambiar el tipo de dato de la columna 'fecha' a datetime
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'], format='%Y-%m-%d', errors='coerce')
df_fleca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309315 entries, 0 to 309325
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           309315 non-null  datetime64[ns]
 1   familia         309315 non-null  object        
 2   base_imponible  309315 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 9.4+ MB


In [73]:
# Ver que el DataFrame tiene los rangos de fecha de datos y los tipos de datos adecuados
df_fleca.describe(include='all')

,fecha,familia,base_imponible
count,309315,309315,309315.000000
unique,NaN,12,NaN
top,NaN,CAFES,NaN
freq,NaN,120703,NaN
mean,2024-03-11 19:50:07.966635776,NaN,2.024148
min,2023-01-02 00:00:00,NaN,0.000000
25%,2023-07-16 00:00:00,NaN,1.360000
50%,2024-04-15 00:00:00,NaN,1.640000
75%,2024-10-16 00:00:00,NaN,2.320000
max,2025-06-30 00:00:00,NaN,287.270000


In [74]:
# Guardar el DataFrame transformado en la carpeta interim
df_fleca.to_parquet('../data/interim/validated_range_dates_20250630.parquet', index=False)